In [1]:
#based on scripts in https://github.com/gschivley/PowerGenome/tree/master/notebooks
#last modified: Aranya Venkatesh, Oct 14, 2020

%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter("ignore")

from pathlib import Path
import geopandas as gpd
import numpy as np
import pandas as pd
from powergenome.generators import GeneratorClusters, add_genx_model_tags
from powergenome.GenX import reduce_time_domain
from powergenome.load_profiles import make_final_load_curves
from powergenome.params import DATA_PATHS
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
)
from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

from powergenome.load_profiles import (
    make_load_curves, 
    add_load_growth, 
    make_final_load_curves, 
    make_distributed_gen_profiles,
)
from powergenome.external_data import make_demand_response_profiles
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    reverse_dict_of_lists,
    remove_feb_29
)

from powergenome.load_profiles import make_final_load_curves
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    reverse_dict_of_lists
)

from powergenome.GenX import reduce_time_domain, add_misc_gen_values
from powergenome.external_data import make_generator_variability

from powergenome.generators import load_ipm_shapefile
from powergenome.GenX import (
    network_line_loss,
    network_max_reinforcement,
    network_reinforcement_cost,
)
from powergenome.transmission import (
    agg_transmission_constraints,
    transmission_line_distance,
)
from powergenome.util import init_pudl_connection, load_settings

pd.options.display.max_columns = 200

In [2]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt

In [3]:
run_folder = "US_National"
settings_file = "US_N_settings.yml"
scenario = "p6"

In [5]:
pudl_engine, pudl_out = init_pudl_connection()
cwd = Path.cwd()

settings_path = (
    cwd / run_folder / settings_file
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

In [6]:
all_periods = list(scenario_settings.keys())
start_year = all_periods[0]
run_new = 1
file_prefix = str(settings_path).replace('.yml','_')

In [ ]:
if run_new==1:
    new_gen = pd.DataFrame()
    for year in all_periods[1:]: #new gen for start year is included in all_gen
        gc = GeneratorClusters(pudl_engine, pudl_out, scenario_settings[year][scenario])
        new_gen_year = gc.create_new_generators()
        new_gen_year.loc[:,'operating_year'] = year
        new_gen = pd.concat([new_gen, new_gen_year]) #create new generators for periods beyond the first

    load_curves = make_final_load_curves(pudl_engine, scenario_settings[start_year][scenario])

    gc = GeneratorClusters(pudl_engine, pudl_out, scenario_settings[start_year][scenario])
    all_gens = gc.create_all_generators() #create existing and new generators for the first time period
    
    #add misc_values from misc_gen_inputs_fn file in extra_inputs folder
    all_gens = add_misc_gen_values(all_gens,settings)
    new_gen = add_misc_gen_values(new_gen,settings)

    gen_variability = make_generator_variability(all_gens)

    (
        reduced_resource_profile,
        reduced_load_profile,
        long_duration_storage,
    ) = reduce_time_domain(gen_variability, load_curves, scenario_settings[start_year][scenario])
    
    if len(settings['region_aggregations'])>1:
        transmission = agg_transmission_constraints(pudl_engine=pudl_engine, settings=settings)
        model_regions_gdf = load_ipm_shapefile(settings)
        transmission = transmission_line_distance(
            trans_constraints_df=transmission,
            ipm_shapefile=model_regions_gdf,
            settings=settings,
        )
        transmission = network_line_loss(transmission=transmission, settings=settings)
        transmission = network_reinforcement_cost(transmission=transmission, settings=settings)
        transmission = network_max_reinforcement(transmission=transmission, settings=settings)
        transmission.to_csv(file_prefix + 'transmission.csv', index=False)

    reduced_load_profile.to_csv(file_prefix + 'reduced_load_profile.csv', index=False)
    reduced_resource_profile.to_csv(file_prefix + 'reduced_resource_profile.csv', index=False)
    all_gens.to_csv(file_prefix + 'all_gens.csv', index=False)
    new_gen.to_csv(file_prefix + 'new_gen.csv', index=False)


961.5000000000002  MW without lat/lon


In [ ]:
if run_new!=1:
    reduced_load_profile = pd.read_csv(file_prefix + 'reduced_load_profile.csv')
    reduced_resource_profile = pd.read_csv(file_prefix + 'reduced_resource_profile.csv')
    all_gens = pd.read_csv(file_prefix + 'all_gens.csv')
    new_gen = pd.read_csv(file_prefix + 'new_gen.csv')
    if len(settings['region_aggregations'])>1:
        transmission = pd.read_csv(file_prefix + 'transmission.csv')

In [ ]:
#create unique technology with resource, region and cluster name
all_gens.loc[:, 'tech'] =  all_gens.loc[:, 'Resource'] + '-' + all_gens.loc[:,'region'] + '-' +all_gens.loc[:,'cluster'].map(int).map(str)
#concat start year gens with remaining new gens
all_gens_multi_year = pd.concat([all_gens, new_gen])
all_gens_multi_year = all_gens_multi_year[all_gens.columns]
all_gens_multi_year.loc[np.isnan(all_gens_multi_year.operating_year),'operating_year'] = all_periods[0]
all_gens_multi_year.loc[all_gens_multi_year.operating_year==0,'operating_year'] = all_periods[0]